在这部分的练习中，你将建立一个逻辑回归模型来预测一个学生是否能进入大学。假设你是一所大学的行政管理人员，你想根据两门考试的结果，来决定每个申请人是否被录取。你有以前申请人的历史数据，可以将其用作逻辑回归训练集。对于每一个训练样本，你有申请人两次测评的分数以及录取的结果。为了完成这个预测任务，我们准备构建一个可以基于两次测试评分来评估录取可能性的分类模型。


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
data = pd.read_csv('ex2data1.txt', names=['exam1', 'exam2', 'admitted'])
data.head()

In [3]:
positive = data[data.admitted.isin(['1'])]  # 1
negetive = data[data.admitted.isin(['0'])]  # 0

fig, ax = plt.subplots(figsize=(6,5))
ax.scatter(positive['exam1'], positive['exam2'], c='b', label='Admitted')
ax.scatter(negetive['exam1'], negetive['exam2'], s=50, c='r', marker='x', label='Not Admitted')
# 设置图例显示在图的上方
box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width , box.height* 0.8])
ax.legend(loc='center left', bbox_to_anchor=(0.2, 1.12),ncol=3)
# 设置横纵坐标名
ax.set_xlabel('Exam 1 Score')
ax.set_ylabel('Exam 2 Score')
plt.show()

NameError: name 'data' is not defined

In [4]:
def sigmoid(z):
    return 1/(1+np.exp(-z))

In [5]:
def cost(theta,X,y):
    first=(-y)*np.log(sigmoid(X@theta))
    second=(1-y)*np.log(1-sigmoid(X@theta))
    return np.mean(first-second) #先累加，后平均
# add a ones column - this makes the matrix multiplication work out easier
#先添加一列
if 'Ones' not in data.columns:
    data.insert(0, 'Ones', 1)

# set X (training data) and y (target variable)
X = data.iloc[:, :-1].as_matrix()  # Convert the frame to its Numpy-array representation.
y = data.iloc[:, -1].as_matrix()  # Return is NOT a Numpy-matrix, rather, a Numpy-array.

#在更新theta使得J最小，与第一章类似
theta = np.zeros(X.shape[1])
#输出初始状态下的损失函数，theta为全零
cost(theta, X, y)
# 0.6931471805599453
#批量梯度下降，采用向量法计算
def gradient(theta, X, y):
    return (X.T @ (sigmoid(X @ theta) - y))/len(X)  
# the gradient of the cost is a vector of the same length as θ where the jth element (for j = 0, 1, . . . , n)
gradient(theta, X, y)
# array([ -0.1, -12.00921659, -11.26284221])

#学习好了参数θ后，我们来用这个模型预测某个学生是否能被录取。
def predict(theta, X):
    probability = sigmoid(X@theta)
    return [1 if x >= 0.5 else 0 for x in probability]  # return a list
#当h(x)>0.5时，判定为1，录取，否则，为不录取。
#更新正确率
final_theta=result[0]
predictions=predict(final_theta,X)
correct=[1 if a==b else 0 for(a,b) in zip(predictions,y)]
acc=sum(correct)/len(X)


NameError: name 'data' is not defined

## 决策边界


逻辑回归，可视化，判定为1/0。 
将其划分：X*theta=0


htheta(x)=sigmoid(theta0*1+theta1*x1+theta2*x2)
利用随机梯度下降法，使得loss最小，更新theta值，
预测的时候，是将sigmoid（theta与X相乘），大于0.5，则判定为1，否则，为0
当theta*X=0,即sigmoid为0.5时，划分边界

x1和x2分别为横纵坐标

In [6]:
x1 = np.arange(130, step=0.1)
x2 = -(final_theta[0] + x1*final_theta[1]) / final_theta[2]
fig, ax = plt.subplots(figsize=(8,5))
ax.scatter(positive['exam1'], positive['exam2'], c='b', label='Admitted')
ax.scatter(negetive['exam1'], negetive['exam2'], s=50, c='r', marker='x', label='Not Admitted')
ax.plot(x1, x2)
ax.set_xlim(0, 130)
ax.set_ylim(0, 130)
ax.set_xlabel('x1')
ax.set_ylabel('x2')
ax.set_title('Decision Boundary')
plt.show()


NameError: name 'final_theta' is not defined

## 对于非线性，引入正则化，并且映射到多维，绘制决策边界

无线性的决策界限
我们把这些特征映射到所有的x1和x2的多项式上，直到六次幂


In [7]:
def feature_mapping(x1, x2, power):
    data = {}
    for i in np.arange(power + 1):
        for p in np.arange(i + 1):
            data["f{}{}".format(i - p, p)] = np.power(x1, i - p) * np.power(x2, p)

#     data = {"f{}{}".format(i - p, p): np.power(x1, i - p) * np.power(x2, p)
#                 for i in np.arange(power + 1)
#                 for p in np.arange(i + 1)
#             }
    return pd.DataFrame(data)
x1 = data2['Test 1'].as_matrix()
x2 = data2['Test 2'].as_matrix()

_data2 = feature_mapping(x1, x2, power=6)
_data2.head()


NameError: name 'data2' is not defined

经过映射，我们将有两个特征的向量转化成了一个28维的向量。
在这个高维特征向量上训练的logistic回归分类器将会有一个更复杂的决策边界，当我们在二维图中绘制时，会出现非线性。
虽然特征映射允许我们构建一个更有表现力的分类器，但它也更容易过拟合。在接下来的练习中，我们将实现正则化的logistic回归来拟合数据，并且可以看到正则化如何帮助解决过拟合的问题。

In [8]:
# 这里因为做特征映射的时候已经添加了偏置项，所以不用手动添加了。
X = _data2.as_matrix()  
y = data2['Accepted'].as_matrix()
theta = np.zeros(X.shape[1])
X.shape, y.shape, theta.shape  # ((118, 28), (118,), (28,))


NameError: name '_data2' is not defined

添加正则化项，不添加第一项

In [9]:
def costReg(theta, X, y, l=1):
    # 不惩罚第一项
    _theta = theta[1: ]
    reg = (l / (2 * len(X))) *(_theta @ _theta)  # _theta@_theta == inner product
    
    return cost(theta, X, y) + reg

costReg(theta, X, y, l=1)  #     0.6931471805599454
#损失函数定义

NameError: name 'theta' is not defined

In [10]:
#梯度下降法
#即在theta更新式子后面加上正则化
def gradientReg(theta, X, y, l=1):
    reg = (1 / len(X)) * theta
    reg[0] = 0  
    return gradient(theta, X, y) + reg
gradientReg(theta, X, y, 1)


NameError: name 'theta' is not defined

利用损失函数和梯度下降法加上正则化，开始更新参数

In [11]:
result2 = opt.fmin_tnc(func=costReg, x0=theta, fprime=gradientReg, args=(X, y, 2))
result2
final_theta = result2[0]
predictions = predict(final_theta, X)
correct = [1 if a==b else 0 for (a, b) in zip(predictions, y)]
accuracy = sum(correct) / len(correct)
accuracy


NameError: name 'opt' is not defined

绘制决策边界

In [ ]:
x = np.linspace(-1, 1.5, 250)
xx, yy = np.meshgrid(x, x)

z = feature_mapping(xx.ravel(), yy.ravel(), 6).as_matrix() #高维
z = z @ final_theta
z = z.reshape(xx.shape)

plot_data()
plt.contour(xx, yy, z, 0)
plt.ylim(-.8, 1.2)
